# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

Date      id Warehouse   Shipments  \
0  Sep 29 2022  8:30AM  247519        10  WT5357 Out   
1  Sep 28 2022  5:41PM  247522        15     8585097   
2  Sep 28 2022  5:41PM  247522        15     8585099   
3  Sep 28 2022  5:41PM  247522        15     8585108   
4  Sep 28 2022  1:53PM  247509        10  0086019013   
5  Sep 28 2022  1:53PM  247509        10  0086021247   
6  Sep 28 2022  1:53PM  247509        10  0086021610   
7  Sep 28 2022  1:53PM  247509        10  0086021613   
8  Sep 28 2022  1:53PM  247509        10  0086021638   
9  Sep 28 2022  1:53PM  247509        10  0086022092   

                                Customer ShipmentStatus  
0                       Eywa Pharma Inc.       Released  
1  Carwin Pharmaceutical Associates, LLC       Released  
2  Carwin Pharmaceutical Associates, LLC       Released  
3  Carwin Pharmaceutical Associates, LLC       Released  
4                      ISDIN Corporation      Completed  
5                      ISDIN Corporation      Completed  
6                      ISDIN Corporation      Completed  
7                      ISDIN Corporation      Completed  
8                      ISDIN Corporation      Completed  
9                      ISDIN Corporation      Completed

In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

id ShipmentStatus  Shipments
4  247494       Released          1
5  247509      Completed         10
6  247509       Released          1
7  247519       Released          1
8  247522       Released          3

In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus  Completed  Released
id                                 
247493                NaN       1.0
247494                NaN       1.0
247509               10.0       1.0
247519                NaN       1.0
247522                NaN       3.0

In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus  Completed  Released
id                                 
247471                0.0       1.0
247480                0.0       1.0
247492                0.0       1.0
247493                0.0       1.0
247494                0.0       1.0

In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

Column Executing does not exist yet


ShipmentStatus  Completed  Released
id                                 
247471                  0         1
247480                  0         1
247492                  0         1
247493                  0         1
247494                  0         1

In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus      id  Completed  Released
0               247471          0         1
1               247480          0         1
2               247492          0         1
3               247493          0         1
4               247494          0         1

In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

Column Executing does not exist yet


ShipmentStatus      id Completed  Released
0               247471                   1
1               247480                   1
2               247492                   1
3               247493                   1
4               247494                   1

In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Released      int32
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

Date      id Warehouse  \
0   Sep 29 2022  8:30AM  247519        10   
1   Sep 28 2022  5:41PM  247522        15   
4   Sep 28 2022  1:53PM  247509        10   
15  Sep 28 2022 12:11PM  247494        22   
16  Sep 28 2022 12:11PM  247493        22   
17  Sep 28 2022 12:09PM  247492        22   
18  Sep 28 2022 11:02AM  247480        16   
19  Sep 28 2022 10:35AM  247471        10   

                                 Customer  
0                        Eywa Pharma Inc.  
1   Carwin Pharmaceutical Associates, LLC  
4                       ISDIN Corporation  
15                      NBTY Global, Inc.  
16                      NBTY Global, Inc.  
17                      NBTY Global, Inc.  
18         Sartorius Bioprocess Solutions  
19                         Citieffe, Inc.

In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

Date      id Warehouse  \
0  Sep 29 2022  8:30AM  247519        10   
1  Sep 28 2022  5:41PM  247522        15   
2  Sep 28 2022  1:53PM  247509        10   
3  Sep 28 2022 12:11PM  247494        22   
4  Sep 28 2022 12:11PM  247493        22   
5  Sep 28 2022 12:09PM  247492        22   
6  Sep 28 2022 11:02AM  247480        16   
7  Sep 28 2022 10:35AM  247471        10   

                                Customer Completed  Released  
0                       Eywa Pharma Inc.                   1  
1  Carwin Pharmaceutical Associates, LLC                   3  
2                      ISDIN Corporation        10         1  
3                      NBTY Global, Inc.                   1  
4                      NBTY Global, Inc.                   1  
5                      NBTY Global, Inc.                   1  
6         Sartorius Bioprocess Solutions                   1  
7                         Citieffe, Inc.                   1

In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

Date      id Warehouse  \
0  Sep 29 2022  8:30AM  247519        10   
1  Sep 28 2022  5:41PM  247522        15   
2  Sep 28 2022  1:53PM  247509        10   
3  Sep 28 2022 12:11PM  247494        22   
4  Sep 28 2022 12:11PM  247493        22   
5  Sep 28 2022 12:09PM  247492        22   
6  Sep 28 2022 11:02AM  247480        16   
7  Sep 28 2022 10:35AM  247471        10   

                                Customer  Released Completed  
0                       Eywa Pharma Inc.         1            
1  Carwin Pharmaceutical Associates, LLC         3            
2                      ISDIN Corporation         1        10  
3                      NBTY Global, Inc.         1            
4                      NBTY Global, Inc.         1            
5                      NBTY Global, Inc.         1            
6         Sartorius Bioprocess Solutions         1            
7                         Citieffe, Inc.         1

In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

Date      id Warehouse  \
0  Sep 29 2022  8:30AM  247519        10   
1  Sep 28 2022  5:41PM  247522        15   
2  Sep 28 2022  1:53PM  247509        10   
3  Sep 28 2022 12:11PM  247494        22   
4  Sep 28 2022 12:11PM  247493        22   

                                Customer  Released Completed  
0                       Eywa Pharma Inc.         1            
1  Carwin Pharmaceutical Associates, LLC         3            
2                      ISDIN Corporation         1        10  
3                      NBTY Global, Inc.         1            
4                      NBTY Global, Inc.         1

In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

Column Executing does not exist yet


Date      id Warehouse  \
0  Sep 29 2022  8:30AM  247519        10   
1  Sep 28 2022  5:41PM  247522        15   
2  Sep 28 2022  1:53PM  247509        10   
3  Sep 28 2022 12:11PM  247494        22   
4  Sep 28 2022 12:11PM  247493        22   

                                Customer  Released  Completed  
0                       Eywa Pharma Inc.         1        NaN  
1  Carwin Pharmaceutical Associates, LLC         3        NaN  
2                      ISDIN Corporation         1       10.0  
3                      NBTY Global, Inc.         1        NaN  
4                      NBTY Global, Inc.         1        NaN

In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

Date      id Warehouse  \
0  Sep 29 2022  8:30AM  247519        10   
1  Sep 28 2022  5:41PM  247522        15   
2  Sep 28 2022  1:53PM  247509        10   
3  Sep 28 2022 12:11PM  247494        22   
4  Sep 28 2022 12:11PM  247493        22   

                                Customer  Released  Completed  
0                       Eywa Pharma Inc.         1        0.0  
1  Carwin Pharmaceutical Associates, LLC         3        0.0  
2                      ISDIN Corporation         1       10.0  
3                      NBTY Global, Inc.         1        0.0  
4                      NBTY Global, Inc.         1        0.0

In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

id  Released  Completed
Warehouse                             
10         742499         3       10.0
15         247522         3        0.0
16         247480         1        0.0
22         742479         3        0.0

In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

Warehouse      id  Released  Completed
0        10  742499         3       10.0
1        15  247522         3        0.0
2        16  247480         1        0.0
3        22  742479         3        0.0

In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

Warehouse  Released  Completed
0        10         3       10.0
1        15         3        0.0
2        16         1        0.0
3        22         3        0.0

In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

Warehouse     Status  Value
0        10   Released    3.0
1        15   Released    3.0
2        16   Released    1.0
3        22   Released    3.0
4        10  Completed   10.0

In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse    10   15   16   22
Status                        
Completed  10.0  0.0  0.0  0.0
Released    3.0  3.0  1.0  3.0

In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse     Status    10   15   16   22
0          Completed  10.0  0.0  0.0  0.0
1           Released   3.0  3.0  1.0  3.0

In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

Status    10   15   16   22
0  Completed  10.0  0.0  0.0  0.0
1   Released   3.0  3.0  1.0  3.0

In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()